<a href="https://colab.research.google.com/github/spirosChv/smartNetsWorkshop/blob/main/neuron/practical3_part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simulating dendrites - Part 2: Segregated dendrites

In [ ]:
!pip install neuron --quiet

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from neuron import h
h.load_file("stdrun.hoc")

In [ ]:
# @title Make nicer plots -- Execute this cell
def mystyle():
  """
  Create custom plotting style.

  Returns
  -------
  my_style : dict
      Dictionary with matplotlib parameters.

  """
  # color pallette
  style = {
      # Use LaTeX to write all text
      "text.usetex": False,
      "font.family": "DejaVu Sans",
      "font.weight": "bold",
      # Use 16pt font in plots, to match 16pt font in document
      "axes.labelsize": 16,
      "axes.titlesize": 20,
      "font.size": 16,
      # Make the legend/label fonts a little smaller
      "legend.fontsize": 14,
      "xtick.labelsize": 14,
      "ytick.labelsize": 14,
      "axes.linewidth": 2.5,
      "lines.markersize": 10.0,
      "lines.linewidth": 2.5,
      "xtick.major.width": 2.2,
      "ytick.major.width": 2.2,
      "axes.labelweight": "bold",
      "axes.spines.right": False,
      "axes.spines.top": False
  }

  return style


plt.style.use("seaborn-colorblind")
plt.rcParams.update(mystyle())

In [ ]:
# Simulation parameters	
tstop = 100  # ms
h.dt = 0.1  # ms
vinit = -65  # initial membrane potential

In [ ]:
# create five compartments using the command `h.Section()`
soma = h.Section(name='soma')
dend0, dend1, dend2, dend3 = [h.Section(name=n) for n in ['dend0', 'dend1', 'dend2', 'dend3']]

# Define properties of soma
soma.diam = 20  # diameter (um) (micro-meter)
soma.L = 200  # length (um)
soma.Ra = 100  # Axial resistance in Ohm * cm
soma.cm = 1  # specific membrane capacitance, capacitance per unit of membrane, uF/cm^2   
soma.nseg = 1  # number of segments (spatial grid)

# Insert Hodgkin-Huxley channels (including leak)
soma.insert('hh')
for seg in soma: 
  seg.hh.gnabar = 0.05  # Sodium conductance in S/cm2
  seg.hh.gkbar = 0.036  # Potassium conductance in S/cm2
  seg.hh.gl = 0.0003  # Leak conductance in S/cm2
  seg.hh.el = -65  # Reversal potential in mV

# Define properties of dend0
dend0.diam = 5
dend0.L = 500
dend0.nseg = 11
dend0.cm = 1
dend0.Ra = 100

dend0.insert('pas')  # Insert passive (leak) channels
for seg in dend0:
  seg.pas.e= -65
  seg.pas.g= 0.0003

# Define properties of dend1
dend1.diam = 3
dend1.L = 200
dend1.nseg = 13
dend1.cm = 1
dend1.Ra = 100

dend1.insert('pas')  # Insert passive (leak) channels
for seg in dend1:
  seg.pas.e = -65
  seg.pas.g = 0.0003
    
# Define properties of dend0
dend2.diam = 5
dend2.L = 500
dend2.nseg = 11
dend2.cm = 1
dend2.Ra = 100

dend2.insert('pas')  # Insert passive (leak) channels
for seg in dend2:
  seg.pas.e = -65
  seg.pas.g = 0.0003

# Define properties of dend1
dend3.diam = 3
dend3.L = 200
dend3.nseg = 13
dend3.cm = 1
dend3.Ra = 100

dend3.insert('pas')  # Insert passive (leak) channels
for seg in dend3:
  seg.pas.e = -65
  seg.pas.g = 0.0003
    
# Connect the compartments together
dend0.connect(soma(0), 0)  # Connect the `0` point of dend0 to the `0` point of the soma
dend1.connect(dend0(1), 0)  # Connect the `0` point of dend1 to the `1` point of the dend0
dend2.connect(soma(1), 0)  # Connect the `0` point of dend2 to the `1` point of the soma
dend3.connect(dend2(1), 0)  # Connect the `0` point of dend3 to the `1` point of the dend2

In [ ]:
#========== Synaptic stimulation.
# Place the synapse to the middle (0.5) of the soma, ...
syn0 = h.ExpSyn(dend1(0.5))
syn0.e = 0  # Reverse potential of the synapse (mV)

# Place the synapse to the middle (0.5) of the soma, ...
syn1 = h.ExpSyn(dend3(0.5))
syn1.e = 0

#========== ...create an artificial spike (an "event" to be delivered to the synapse)...
tsignal = 50
ns = h.NetStim(0.5)
ns.start = tsignal
ns.number = 1

#... and connect the event to the synapse.
nc0 = h.NetCon(ns, syn0)
nc0.delay = 0

nc1 = h.NetCon(ns, syn1)
nc1.delay = 0

### Task 1: We will add one `h.ExpSyn` in the middle of `dend1` and one in the middle of `dend3`. 
### Activate (`weight*3`) each one of them and both of them. Is the response the linear sum?

**Hint!**

Try different combinations of `nc0.weight[0]` and `nc1.weight[0]`

 - Activate `nc1` instead of `nc0`.
 - Activate both `nc0` and `nc1`.

In [ ]:
nc0.weight[0] = 0.008*3
nc1.weight[0] = 0.008*0

# Calculate and print in terminal the depolarization at the soma and at the middle part of the distal dendrite (dend3)
vsoma_vec = h.Vector().record(soma(0.5)._ref_v)  # Membrane potential vector
t_vec = h.Vector().record(h._ref_t)  # Time stamp vector

# reinitialize the simulator and run again
h.finitialize(vinit)
h.continuerun(tstop)

# Remove the first 20ms to avoid artifacts
tremove = 20
vsoma_vec.remove(0, int(tremove/h.dt))
t_vec.remove(0, int(tremove/h.dt))

plt.figure(figsize=(8, 6))
plt.plot(t_vec, vsoma_vec, color='black', label='soma')
plt.legend()
plt.xlabel('time (ms)')
plt.ylabel('mV')
plt.show()

print(f'\nSomatic depolarization is {np.round(vsoma_vec.max() - vsoma_vec[int((tsignal-tremove)/h.dt)-1], 2)} mV')